# NPLM的实现


## 1.数据准备

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import jieba
import re
import numpy as np
from sklearn.decomposition import PCA
import matplotlib
import matplotlib.pyplot as plt



In [4]:
f = open("swarma_article_small.txt", 'r', encoding='utf-8')
text = str(f.read())
f.close()

temp = jieba.lcut(text)
words = []
for i in temp:
    #过滤掉所有的标点符号
    i = re.sub("[\s+\.\!\/_,$%^*(+\"\'“”《》?“]+|[+——！，。？、~@#￥%……&*（）：]+", "", i)
    if len(i) > 0:
        words.append(i)

print(len(words))
words

<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
C:\Users\xiaogang\AppData\Local\Temp\ipykernel_9992\3451816243.py:9: SyntaxWarning: invalid escape sequence '\s'
  i = re.sub("[\s+\.\!\/_,$%^*(+\"\'“”《》?“]+|[+——！，。？、~@#￥%……&*（）：]+", "", i)
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\xiaogang\AppData\Local\Temp\jieba.cache
Loading model cost 1.455 seconds.
Prefix dict has been built successfully.


45321


['导语',
 '科学家',
 '什么',
 '时候',
 '能够',
 '迎来',
 '事业',
 '巅峰',
 '科学',
 '创新',
 '的',
 '生命周期',
 '是',
 '多久',
 '职业生涯',
 '中',
 '出现',
 '突破性',
 '进展',
 '的',
 '迹象',
 '是否',
 '存在',
 '什么样',
 '的',
 '合作',
 '会',
 '带来',
 '成功',
 '年轻',
 '的',
 '科研人员',
 '如何',
 '将',
 '成功',
 '概率',
 '最大化',
 '这些',
 '都',
 '是',
 '科学学',
 'the',
 'science',
 'of',
 'science',
 '正在',
 '探索',
 '的',
 '问题',
 '当',
 '我们',
 '把',
 '科学',
 '视为',
 '一个',
 '复杂',
 '系统',
 '来',
 '研究',
 '会',
 '有',
 '令人兴奋',
 '的',
 '洞察',
 '由',
 '王大顺',
 '美国西北大学',
 '凯洛',
 '格',
 '商学院',
 '教授',
 '科学',
 '与',
 '创新',
 '科学',
 '中心',
 '创始',
 '主任',
 '和',
 'Albert',
 '-',
 'L',
 'á',
 'szl',
 'ó',
 'Barab',
 'á',
 'si',
 '美国',
 '东北大学',
 '复杂',
 '网络',
 '科学',
 '中心',
 '主任',
 '无',
 '标度',
 '网络',
 '模型',
 '提出',
 '者',
 '合作',
 '完成',
 '的',
 '新书',
 '给',
 '科学家',
 '的',
 '科学',
 '思维',
 'The',
 'Science',
 'of',
 'Science',
 '中文版',
 '于',
 '近日',
 '上市',
 '本书',
 '中译',
 '版',
 '由',
 '贾韬',
 '西南',
 '大学',
 '计算机',
 '与',
 '信息科学',
 '学院',
 '教授',
 '和',
 '汪小帆',
 '长江',
 '学者',
 '特聘',
 '教授',
 '上海大学',
 '副校长',
 '完成',
 '翻

In [5]:
# 构建三元组，形成训练数据
# N-gram模型，N=2
trigrams = [([words[i], words[i+1]], words[i+2]) for i in range(len(words)-2)]
print(trigrams[:3])

[(['导语', '科学家'], '什么'), (['科学家', '什么'], '时候'), (['什么', '时候'], '能够')]


In [12]:
# 建立词典
vocab = set(words)
print(len(vocab))
word_to_idx = {}
idx_to_word = {}
ids = 0
for w in words:
    cnt = word_to_idx.get(w, [ids, 0])
    if cnt[1] == 0:
        ids += 1
    cnt[1] += 1
    word_to_idx[w] = cnt
    idx_to_word[ids] = w
word_to_idx

7395


{'导语': [0, 23],
 '科学家': [1, 45],
 '什么': [2, 17],
 '时候': [3, 5],
 '能够': [4, 52],
 '迎来': [5, 3],
 '事业': [6, 4],
 '巅峰': [7, 2],
 '科学': [8, 196],
 '创新': [9, 39],
 '的': [10, 3250],
 '生命周期': [11, 6],
 '是': [12, 375],
 '多久': [13, 4],
 '职业生涯': [14, 7],
 '中': [15, 380],
 '出现': [16, 47],
 '突破性': [17, 4],
 '进展': [18, 15],
 '迹象': [19, 2],
 '是否': [20, 15],
 '存在': [21, 49],
 '什么样': [22, 5],
 '合作': [23, 15],
 '会': [24, 112],
 '带来': [25, 21],
 '成功': [26, 16],
 '年轻': [27, 8],
 '科研人员': [28, 8],
 '如何': [29, 58],
 '将': [30, 120],
 '概率': [31, 9],
 '最大化': [32, 6],
 '这些': [33, 98],
 '都': [34, 82],
 '科学学': [35, 15],
 'the': [36, 45],
 'science': [37, 8],
 'of': [38, 85],
 '正在': [39, 16],
 '探索': [40, 42],
 '问题': [41, 99],
 '当': [42, 32],
 '我们': [43, 180],
 '把': [44, 10],
 '视为': [45, 8],
 '一个': [46, 162],
 '复杂': [47, 284],
 '系统': [48, 361],
 '来': [49, 66],
 '研究': [50, 300],
 '有': [51, 108],
 '令人兴奋': [52, 3],
 '洞察': [53, 2],
 '由': [54, 59],
 '王大顺': [55, 4],
 '美国西北大学': [56, 2],
 '凯洛': [57, 2],
 '格': [58, 3],
 '商学

## 建立模型

In [10]:
class NGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size): # embedding_dim是词向量维数，context_size是窗口大小
        super(NGram, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        # input的尺寸为1*context_size
        embeds = self.embeddings(inputs)
        # embeds的尺寸为context_size*embedding_dim
        embeds = embeds.view(1, -1)
        # embeds的尺寸为1*(context_size*embedding_dim)
        out = self.linear1(embeds)
        out = F.relu(out)

        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs
    def extract(self, inputs):
        embeds = self.embeddings(inputs)
        return embeds
    

## 训练模型

In [15]:
losses = []
criterion = nn.NLLLoss()
model = NGram(len(vocab), 10, 2)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(20):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
        context_idxs = [word_to_idx[w][0] for w in context]
        context_var = torch.LongTensor(context_idxs)
        optimizer.zero_grad()
        log_probs = model(context_var)
        loss = criterion(log_probs, torch.LongTensor(word_to_idx[target][:1]))
        loss.backward()
        optimizer.step()
        total_loss += loss.data
    losses.append(total_loss)
    print('第{}轮，损失函数值为：{:.2f}'.format(epoch, total_loss.numpy()[0]))

第0轮，损失函数值为：373324.28
第1轮，损失函数值为：344477.44
第2轮，损失函数值为：332019.72
第3轮，损失函数值为：325162.09
第4轮，损失函数值为：320337.22
第5轮，损失函数值为：316359.12
第6轮，损失函数值为：312759.91
第7轮，损失函数值为：309337.53
第8轮，损失函数值为：305988.38
第9轮，损失函数值为：302645.94
第10轮，损失函数值为：299296.19
第11轮，损失函数值为：295911.12
第12轮，损失函数值为：292485.44
第13轮，损失函数值为：289006.59
第14轮，损失函数值为：285470.88
第15轮，损失函数值为：281874.53
第16轮，损失函数值为：278216.94
第17轮，损失函数值为：274501.47
第18轮，损失函数值为：270732.34
第19轮，损失函数值为：266913.34


In [ ]:
vec = model.extract(torch.LongTensor([v[0] for v in word_to_idx.values()]))
vec = vec.data.numpy()
X_reduced = PCA(n_component=2).fit_transform(vec)
fig = plt.figure(figsize)